In [1]:
import random
from dataclasses import dataclass, field
from typing import List, Dict, Any

import numpy as np
import os
from tampura.environment import TampuraEnv
from tampura.spec import ProblemSpec
from tampura.structs import (
    AbstractBelief,
    ActionSchema,
    StreamSchema,
    AliasStore,
    Belief,
    NoOp,
    Predicate,
    State,
    effect_from_execute_fn,
    Observation
)
import logging 
from tampura.symbolic import OBJ, Or, Atom, ForAll, Exists, OneOf, And, Not, Eq, negate
from tampura.policies.tampura_policy import TampuraPolicy
from tampura.config.config import get_default_config, setup_logger


PICK_SUCCESS = 0.7
DROP_SUCCESS = 0.8
PICK_BREAK_PROB = 0.5
DROP_BREAK_PROB = 0.7
OBSERVATION_PROB = 0.8
OBJECTS = {f"{OBJ}o1":{"region":1,"shape":"cube","material":"glass"},f"{OBJ}o2":{"region":2,"shape":"sphere","material":"glass"}}#,f"{OBJ}o3":{"region":3,"shape":"sphere","material":"paper"}}
MATERIALS = ["glass","paper"]
DESIRED_SHAPE = "cube"
SHAPES = ["cube","sphere","pyramid"]
REGIONS =  {"robot":4,"goal":5}
THRESH = 0.5

In [12]:
# Observation space


@dataclass 
class ObjectObservation(Observation):
    
    at: Dict[str,str]=field(default_factory=lambda: {})
    shape_probs: Dict[str,List[float]]=field(default_factory=lambda:[])
    shapes: Dict[str,str]=field(default_factory=lambda: [])
    materials: Dict[str,str]=field(default_factory=lambda: {})
    confident: List[str]=field(default_factory=lambda: [])

# Belief space
class ObjectsBelief(Belief):
    def __init__(self, at={}, shape_probs={}, shapes={}, materials={}, confident=[]):
        
        self.at=at
        self.shape_probs=shape_probs
        self.shapes=shapes
        self.materials=materials
        self.confident=confident
        
        
            

    def update(self, a, o, s):
        
        at=o.at
        shape_probs=o.shape_probs
        shapes=o.shapes
        materials=o.materials
        confident=o.confident
        
        return ObjectsBelief(at=at,shape_probs=shape_probs,shapes=shapes,materials=materials,confident=confident) 

    def abstract(self, store: AliasStore):
        
        atoms = []
        
        for o in self.at.keys():
            atoms.append(Atom("at",[o,self.at[o]]))
            
        for o in self.shapes.keys():
            atoms.append(Atom("is_shape",[o,self.shapes[o]]))
            atoms.append(Atom("is_material",[o,self.materials[o]]))
            # atoms.append(Atom(self.shapes[o], [o]))
            # atoms.append(Atom(self.materials[o], [o]))
        
        for o in self.confident:
            atoms.append(Atom("confident", [o]))
            
        return AbstractBelief(atoms)
    

    def vectorize(self):
        pass
    
# Sample function for stream schema
def sample_location(input_sym, store):
    
    o = input_sym[0]
    
    r = OBJECTS[o]["region"]
    
    return r
    
def sample_grasp(input_sym, store):
    
    g = random.random()
    
    # TODO: make more sophisticated
    
    return g

    
# Action simulators

    
def inspect_execute_fn(a, b, s, store):
    
    o = a.args[0]    
    
    at=b.at.copy()
    shape_probs=b.shape_probs.copy()
    shapes=b.shapes.copy()
    materials=b.materials.copy()
    confident=b.confident.copy()
    
    
    # Noisy detection
    actual_shape = OBJECTS[o]["shape"]
    material = OBJECTS[o]["material"]
    
    obs_prob = random.uniform(OBSERVATION_PROB,1.0)
    
    if shape_probs.get(o) is None:
        shape_probs[o] = [1.0/len(SHAPES)]*len(SHAPES)
   
    
    for i,shape in enumerate(SHAPES):
        if shape == actual_shape:
            shape_probs[o][i] *= obs_prob
        else:
            shape_probs[o][i] *= (1-obs_prob)
    
    # normalise
    total = sum(shape_probs[o])
    shape_probs[o] = [p/total for p in shape_probs[o]] 
    
    max_prob = max(shape_probs[o])
    max_idx = shape_probs[o].index(max_prob)
    max_prob_shape = SHAPES[max_idx]
    
    
    if max_prob > THRESH and shapes.get(o) is None:
        confident = list(set(confident+[o]))
        shapes[o] = max_prob_shape
        materials[o] = material # set material only when shape is known for certain
        
    
    return State(), ObjectObservation(at=at,shape_probs=shape_probs,shapes=shapes,materials=materials,confident=confident)

def pick_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]
    
    at=b.at.copy()
    shape_probs=b.shape_probs.copy()
    shapes=b.shapes.copy()
    materials=b.materials.copy()
    confident=b.confident.copy()
    
    if shapes[o] == DESIRED_SHAPE:
        
    
        if random.random() < PICK_SUCCESS:
            
            # if materials[o] == "glass":
            #     if random.random() < PICK_BREAK_PROB:
            #         broken = list(set(broken+[o]))

            
            at[o]="robot" # may break but at desired location
        
    

    return State(), ObjectObservation(at=at,shape_probs=shape_probs,shapes=shapes,materials=materials,confident=confident)
    

def drop_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]
    r = a.args[2]
    
    at=b.at.copy()
    shape_probs=b.shape_probs.copy()
    shapes=b.shapes.copy()
    materials=b.materials.copy()
    confident=b.confident.copy()
    
    
    
    
    if random.random() < DROP_SUCCESS:
        
        # if materials[o] == "glass":
        #     if random.random() < DROP_BREAK_PROB:
        #         broken = list(set(broken+[o]))
        #         # at[o] = "unk" # unknown

        at[o] = r # may break but at desired location
        
    

    return State(), ObjectObservation(at=at,shape_probs=shape_probs,shapes=shapes,materials=materials,confident=confident)

# closed world assumption: not explicitly True predicates are false

# Set up environment dynamics
class ToyDiscrete(TampuraEnv):
    def initialize(self):
        
        store = AliasStore()
        
        for o in OBJECTS.keys():
            store.set(o,o,"physical")
            
        for region in REGIONS.keys():
            store.set(region,REGIONS[region],"region")
            
        for shape in SHAPES:
            store.set(shape,shape,"shape")
            
        for material in MATERIALS:
            store.set(material,material,"material")
        
        return ObjectsBelief(), store

    def get_problem_spec(self) -> ProblemSpec:
        predicates = [ 
            Predicate("at",["physical","region"]),
            Predicate("grasped",["physical","grasp"]),
            Predicate("confident",["physical"]),
            Predicate("is_shape", ["physical","shape"]),
            Predicate("is_material",["physical","material"])
        ] 
        
        stream_schemas = [
            StreamSchema(
                name="sample-location",
                inputs=["?o1"],
                input_types=["physical"],
                output="?r1",
                output_type="region",
                certified=[Atom("at",["?o1","?r1"])],
                sample_fn=sample_location
                
            ),
            
            StreamSchema(
                name="sample-grasp",
                inputs=["?o1"],
                input_types=["physical"],
                output="?g1",
                output_type="grasp",
                certified=[Atom("grasped",["?o1","?g1"])], # generates a valid grasp g1 for o1
                sample_fn=sample_grasp                
            ),

        ]
        
        action_schemas = [
            
            ActionSchema(
                name="inspect",
                inputs=["?o1"],
                input_types=["physical"],
                preconditions=[Exists(Atom("at",["?o1","?r"]),["?r"],["region"]),Not(Exists(Atom("is_shape",["?o1","?s"]),["?s"],["shape"]))],
                verify_effects=[Atom("is_shape",["?o1",shape])for shape in SHAPES], # TODO: OneOf is not usable!! but cannot do away with it either
                execute_fn=inspect_execute_fn,
                effects_fn=effect_from_execute_fn(inspect_execute_fn),
            ),           
             
            ActionSchema(
                name="pick",
                inputs=["?o1","?g1","?r1"],
                input_types=["physical","grasp","region"],
                preconditions=[Atom("at",["?o1","?r1"]), # ?o1 is in region ?r1
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                            #    Atom("confident",["?o1"]),
                               Atom("is_shape",["?o1",DESIRED_SHAPE]), # ?o1 is of the desired shape
                               Not(Exists(Atom("at",["?o","robot"]),["?o"],["physical"])), # robot is not holding any other object
                               ], 
                verify_effects=[Atom("at", ["?o1","robot"]),Not(Atom("at",["?o1","?r1"]))], # verify effects with oneof is not the ideal way 
                execute_fn=pick_execute_fn,
                effects_fn=effect_from_execute_fn(pick_execute_fn),
            ),
            ActionSchema(
                name="drop",
                inputs=["?o1","?g1","?r1"],
                input_types=["physical","grasp","region"],
                preconditions=[Atom("at",["?o1","robot"]), # robot is holding ?o1
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                               Not(Atom("at",["?o1","?r1"])), # current location is not the same as goal location
                               ], 
                verify_effects=[Atom("at", ["?o1","?r1"]),Not(Atom("at",["?o1","robot"]))], # verify effects with oneof is not the ideal way
                execute_fn=drop_execute_fn,
                effects_fn=effect_from_execute_fn(drop_execute_fn),
            ),
            NoOp(),
        ]
        
        # reward = ForAll(Atom("at",["?o","goal"]) , ["?o"], ["physical"])# WORKS for two cubes (kind of) 
        # reward = ForAll(Or([Atom(shape,["?o"])for shape in SHAPES]),["?o"],["physical"])
        reward = ForAll(Or([And([Atom("is_shape",["?o",DESIRED_SHAPE]),Atom("at",["?o","goal"])])]+[Atom("is_shape",["?o",shape]) for shape in SHAPES if shape != DESIRED_SHAPE]),["?o"],["physical"]) # works for two spheres but not one cube one sphere!!

        spec = ProblemSpec(
            predicates=predicates,
            stream_schemas=stream_schemas,
            action_schemas=action_schemas,
            reward=reward,
        )

        return spec



## Create environment and planner

In [13]:
# Planner
cfg = get_default_config(save_dir=os.getcwd())

# Set some print options to print out abstract belief, action, observation, and reward
cfg["print_options"] = "ab,a,o,r"
cfg["vis_graph"] = True
cfg["flat_sample"] = True
cfg["batch_size"] = 100
cfg["num_samples"] = 1000
cfg["max_steps"] = 20


# Initialize environment
env = ToyDiscrete(config=cfg)
b0, store = env.initialize()

# Set up logger to print info
setup_logger(cfg["save_dir"], logging.INFO)

# Initialize the policy
planner = TampuraPolicy(config = cfg, problem_spec = env.problem_spec)

## Run Planner
Make sure symk is installed (see README) before running the Tampura planner.
With the default settings, the planner should pick both every time.

In [14]:
print(store)
history,store = planner.rollout(env, b0, store)

AliasStore(als={'o_o1': 'o_o1', 'o_o2': 'o_o2', 'robot': 4, 'goal': 5, 'cube': 'cube', 'sphere': 'sphere', 'pyramid': 'pyramid', 'glass': 'glass', 'paper': 'paper'}, als_type={'o_o1': 'physical', 'o_o2': 'physical', 'robot': 'region', 'goal': 'region', 'cube': 'shape', 'sphere': 'shape', 'pyramid': 'shape', 'glass': 'material', 'paper': 'material'}, alph_counter={}, certified=[], sample_counts={}, branching_factor={})

==========t=0==========
Abstract Belief: AbstractBelief(items=[])
Reward: 0.0
Flat Stream Sampling
Sampling StreamSchema(name='sample-location', inputs=['?o1'], input_types=['physical'], output='?r1', output_type='region', preconditions=[], certified=[Atom(pred_name='at', args=['?o1', '?r1'])], sample_fn=<function sample_location at 0x79551b5cd120>)(['o_o1'])
Sampling StreamSchema(name='sample-location', inputs=['?o1'], input_types=['physical'], output='?r1', output_type='region', preconditions=[], certified=[Atom(pred_name='at', args=['?o1', '?r1'])], sample_fn=<functio

In [15]:
history.actions

[Action(name='inspect', args=['o_o1']),
 Action(name='inspect', args=['o_o2']),
 Action(name='pick', args=['o_o1', 'o_gr_0', 'o_re_0']),
 Action(name='pick', args=['o_o1', 'o_gr_0', 'o_re_0']),
 Action(name='drop', args=['o_o1', 'o_gr_0', 'goal']),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 None]

In [16]:
history.abstract_beliefs

[AbstractBelief(items=[]),
 AbstractBelief(items=[Atom(pred_name='is_material', args=['o_o1', 'glass']), Atom(pred_name='confident', args=['o_o1']), Atom(pred_name='is_shape', args=['o_o1', 'cube'])]),
 AbstractBelief(items=[Atom(pred_name='is_material', args=['o_o1', 'glass']), Atom(pred_name='confident', args=['o_o1']), Atom(pred_name='is_shape', args=['o_o1', 'cube'])]),
 AbstractBelief(items=[Atom(pred_name='is_material', args=['o_o1', 'glass']), Atom(pred_name='confident', args=['o_o1']), Atom(pred_name='is_shape', args=['o_o1', 'cube']), Atom(pred_name='at', args=['o_o1', 'robot'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'goal']), Atom(pred_name='is_material', args=['o_o1', 'glass']), Atom(pred_name='confident', args=['o_o1']), Atom(pred_name='is_shape', args=['o_o1', 'cube'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'goal']), Atom(pred_name='is_material', args=['o_o1', 'glass']), Atom(pred_name='confident', args=['o_o1']), Atom(pred_name=

In [17]:
store

AliasStore(als={'o_o1': 'o_o1', 'robot': 4, 'goal': 5, 'cube': 'cube', 'sphere': 'sphere', 'pyramid': 'pyramid', 'glass': 'glass', 'paper': 'paper', 'o_re_0': 1, 'o_gr_0': 0.4865266401391204}, als_type={'o_o1': 'physical', 'robot': 'region', 'goal': 'region', 'cube': 'shape', 'sphere': 'shape', 'pyramid': 'shape', 'glass': 'material', 'paper': 'material', 'o_re_0': 'region', 'o_gr_0': 'grasp'}, alph_counter={'re': 1, 'gr': 1}, certified=[Atom(pred_name='at', args=['o_o1', 'o_re_0']), Atom(pred_name='grasped', args=['o_o1', 'o_gr_0'])], sample_counts={}, branching_factor={})